<a href="https://colab.research.google.com/github/Sagar-modelling/Pyspark_DataAnalysis/blob/main/pyspark_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xvf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

--2021-07-23 20:42:28--  https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.209.10, 135.181.214.104, 88.99.95.219, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.209.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228834641 (218M) [application/x-gzip]
Saving to: ‘spark-3.1.2-bin-hadoop3.2.tgz.1’

spark-3.1.2-bin-had 100%[===================>] 218.23M  18.4MB/s    in 13s     

2021-07-23 20:42:41 (16.9 MB/s) - ‘spark-3.1.2-bin-hadoop3.2.tgz.1’ saved [228834641/228834641]

spark-3.1.2-bin-hadoop3.2/
spark-3.1.2-bin-hadoop3.2/R/
spark-3.1.2-bin-hadoop3.2/R/lib/
spark-3.1.2-bin-hadoop3.2/R/lib/sparkr.zip
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/worker.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/daemon.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/
spark-

In [32]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

## Dataset 1

In [3]:
from google.colab import files
uploaded = files.upload()

Saving effects-of-covid-19-on-trade-at-14-July-2021-provisional.csv to effects-of-covid-19-on-trade-at-14-July-2021-provisional.csv


In [4]:
import io
import pandas as pd
df = pd.read_csv(io.BytesIO(uploaded['effects-of-covid-19-on-trade-at-14-July-2021-provisional.csv']))
df

,Direction,Year,Date,Weekday,Country,Commodity,Transport_Mode,Measure,Value,Cumulative
0,Exports,2015,01/01/2015,Thursday,All,All,All,$,104000000,104000000
1,Exports,2015,02/01/2015,Friday,All,All,All,$,96000000,200000000
2,Exports,2015,03/01/2015,Saturday,All,All,All,$,61000000,262000000
3,Exports,2015,04/01/2015,Sunday,All,All,All,$,74000000,336000000
4,Exports,2015,05/01/2015,Monday,All,All,All,$,105000000,442000000
...,...,...,...,...,...,...,...,...,...,...
104660,Reimports,2021,10/07/2021,Saturday,All,All,All,$,0,90000000
104661,Reimports,2021,11/07/2021,Sunday,All,All,All,$,0,90000000
104662,Reimports,2021,12/07/2021,Monday,All,All,All,$,0,91000000
104663,Reimports,2021,13/07/2021,Tuesday,All,All,All,$,1000000,92000000


In [15]:
type(df)

pandas.core.frame.DataFrame

In [7]:
#create spark_session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate() #to ceate sparksession practice
spark

In [8]:
!ls

effects-of-covid-19-on-trade-at-14-July-2021-provisional.csv
sample_data
spark-3.1.2-bin-hadoop3.2
spark-3.1.2-bin-hadoop3.2.tgz


In [ ]:
dataset = spark.read.csv('effects-of-covid-19-on-trade-at-14-July-2021-provisional.csv',inferSchema=True, header =True)

#With the inferSchema option is set to true, Databricks will run a pass over the complete file and determine the column names, data types, and nullability

In [13]:
dataset.show()

+---------+----+----------+---------+-------+---------+--------------+-------+---------+----------+
|Direction|Year|      Date|  Weekday|Country|Commodity|Transport_Mode|Measure|    Value|Cumulative|
+---------+----+----------+---------+-------+---------+--------------+-------+---------+----------+
|  Exports|2015|01/01/2015| Thursday|    All|      All|           All|      $|104000000| 104000000|
|  Exports|2015|02/01/2015|   Friday|    All|      All|           All|      $| 96000000| 200000000|
|  Exports|2015|03/01/2015| Saturday|    All|      All|           All|      $| 61000000| 262000000|
|  Exports|2015|04/01/2015|   Sunday|    All|      All|           All|      $| 74000000| 336000000|
|  Exports|2015|05/01/2015|   Monday|    All|      All|           All|      $|105000000| 442000000|
|  Exports|2015|06/01/2015|  Tuesday|    All|      All|           All|      $| 76000000| 518000000|
|  Exports|2015|07/01/2015|Wednesday|    All|      All|           All|      $| 59000000| 577000000|


In [16]:
dataset.printSchema()

root
 |-- Direction: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- Weekday: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Commodity: string (nullable = true)
 |-- Transport_Mode: string (nullable = true)
 |-- Measure: string (nullable = true)
 |-- Value: integer (nullable = true)
 |-- Cumulative: long (nullable = true)



In [14]:
type(dataset)

pyspark.sql.dataframe.DataFrame

# **Data** preprocessing

In [19]:
dataset.select('Direction','Weekday').show()

+---------+---------+
|Direction|  Weekday|
+---------+---------+
|  Exports| Thursday|
|  Exports|   Friday|
|  Exports| Saturday|
|  Exports|   Sunday|
|  Exports|   Monday|
|  Exports|  Tuesday|
|  Exports|Wednesday|
|  Exports| Thursday|
|  Exports|   Friday|
|  Exports| Saturday|
|  Exports|   Sunday|
|  Exports|   Monday|
|  Exports|  Tuesday|
|  Exports|Wednesday|
|  Exports| Thursday|
|  Exports|   Friday|
|  Exports| Saturday|
|  Exports|   Sunday|
|  Exports|   Monday|
|  Exports|  Tuesday|
+---------+---------+
only showing top 20 rows



In [20]:
dataset.describe().show

+-------+---------+------------------+----------+---------+-------------+--------------------+--------------+-------+-------------------+-------------------+
|summary|Direction|              Year|      Date|  Weekday|      Country|           Commodity|Transport_Mode|Measure|              Value|         Cumulative|
+-------+---------+------------------+----------+---------+-------------+--------------------+--------------+-------+-------------------+-------------------+
|  count|   104665|            104665|    104665|   104665|       104665|              104665|        104665| 104665|             104665|             104665|
|   mean|     null|2017.7871590311947|      null|     null|         null|                null|          null|   null|2.863475479864329E7|4.927746986117614E9|
| stddev|     null|1.8970924928457573|      null|     null|         null|                null|          null|   null|5.175916660503251E7|9.378260046496975E9|
|    min|  Exports|              2015|01/01/2015|   

In [22]:
#adding columns
dataset = dataset.withColumn('Added column', dataset['value']+2)

In [23]:
dataset.show()

+---------+----+----------+---------+-------+---------+--------------+-------+---------+----------+------------+
|Direction|Year|      Date|  Weekday|Country|Commodity|Transport_Mode|Measure|    Value|Cumulative|Added column|
+---------+----+----------+---------+-------+---------+--------------+-------+---------+----------+------------+
|  Exports|2015|01/01/2015| Thursday|    All|      All|           All|      $|104000000| 104000000|   104000002|
|  Exports|2015|02/01/2015|   Friday|    All|      All|           All|      $| 96000000| 200000000|    96000002|
|  Exports|2015|03/01/2015| Saturday|    All|      All|           All|      $| 61000000| 262000000|    61000002|
|  Exports|2015|04/01/2015|   Sunday|    All|      All|           All|      $| 74000000| 336000000|    74000002|
|  Exports|2015|05/01/2015|   Monday|    All|      All|           All|      $|105000000| 442000000|   105000002|
|  Exports|2015|06/01/2015|  Tuesday|    All|      All|           All|      $| 76000000| 5180000

In [25]:
#drop columns
dataset = dataset.drop('Added column')

In [26]:
dataset.show()

+---------+----+----------+---------+-------+---------+--------------+-------+---------+----------+
|Direction|Year|      Date|  Weekday|Country|Commodity|Transport_Mode|Measure|    Value|Cumulative|
+---------+----+----------+---------+-------+---------+--------------+-------+---------+----------+
|  Exports|2015|01/01/2015| Thursday|    All|      All|           All|      $|104000000| 104000000|
|  Exports|2015|02/01/2015|   Friday|    All|      All|           All|      $| 96000000| 200000000|
|  Exports|2015|03/01/2015| Saturday|    All|      All|           All|      $| 61000000| 262000000|
|  Exports|2015|04/01/2015|   Sunday|    All|      All|           All|      $| 74000000| 336000000|
|  Exports|2015|05/01/2015|   Monday|    All|      All|           All|      $|105000000| 442000000|
|  Exports|2015|06/01/2015|  Tuesday|    All|      All|           All|      $| 76000000| 518000000|
|  Exports|2015|07/01/2015|Wednesday|    All|      All|           All|      $| 59000000| 577000000|


In [27]:
#rename the columns
dataset = dataset.withColumnRenamed('Country','India')
dataset.show()

+---------+----+----------+---------+-----+---------+--------------+-------+---------+----------+
|Direction|Year|      Date|  Weekday|India|Commodity|Transport_Mode|Measure|    Value|Cumulative|
+---------+----+----------+---------+-----+---------+--------------+-------+---------+----------+
|  Exports|2015|01/01/2015| Thursday|  All|      All|           All|      $|104000000| 104000000|
|  Exports|2015|02/01/2015|   Friday|  All|      All|           All|      $| 96000000| 200000000|
|  Exports|2015|03/01/2015| Saturday|  All|      All|           All|      $| 61000000| 262000000|
|  Exports|2015|04/01/2015|   Sunday|  All|      All|           All|      $| 74000000| 336000000|
|  Exports|2015|05/01/2015|   Monday|  All|      All|           All|      $|105000000| 442000000|
|  Exports|2015|06/01/2015|  Tuesday|  All|      All|           All|      $| 76000000| 518000000|
|  Exports|2015|07/01/2015|Wednesday|  All|      All|           All|      $| 59000000| 577000000|
|  Exports|2015|08/0

# **Handling Missing** **Values**- Dataset 2

In [28]:
from google.colab import files
uploaded = files.upload()

Saving melb_data.csv to melb_data.csv


In [80]:
#create spark_session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate() #to ceate sparksession practice
spark

In [81]:
dataset2 = spark.read.csv('melb_data.csv',inferSchema=True, header =True)

In [82]:
dataset2.show()

+----------+-------------------+-----+----+---------+------+-------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|    Suburb|            Address|Rooms|Type|    Price|Method|SellerG|      Date|Distance|Postcode|Bedroom2|Bathroom|Car|Landsize|BuildingArea|YearBuilt|CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|
+----------+-------------------+-----+----+---------+------+-------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|Abbotsford|       85 Turner St|    2|   h|1480000.0|     S| Biggin| 3/12/2016|     2.5|  3067.0|     2.0|     1.0|1.0|   202.0|        null|     null|      Yarra| -37.7996|  144.9984|Northern Metropol...|       4019.0|
|Abbotsford|    25 Bloomburg St|    2|   h|1035000.0|     S| Biggin| 4/02/2016|     2.5|  3067.0|     2.0|     1.0|0.0| 

In [83]:
dataset2.printSchema()

root
 |-- Suburb: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Rooms: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Method: string (nullable = true)
 |-- SellerG: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Postcode: double (nullable = true)
 |-- Bedroom2: double (nullable = true)
 |-- Bathroom: double (nullable = true)
 |-- Car: double (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- BuildingArea: double (nullable = true)
 |-- YearBuilt: double (nullable = true)
 |-- CouncilArea: string (nullable = true)
 |-- Lattitude: double (nullable = true)
 |-- Longtitude: double (nullable = true)
 |-- Regionname: string (nullable = true)
 |-- Propertycount: double (nullable = true)



In [84]:
dataset2.select('BuildingArea','YearBuilt').show() #missing values

+------------+---------+
|BuildingArea|YearBuilt|
+------------+---------+
|        null|     null|
|        79.0|   1900.0|
|       150.0|   1900.0|
|        null|     null|
|       142.0|   2014.0|
|        null|     null|
|       210.0|   1910.0|
|       107.0|   1890.0|
|        null|     null|
|        75.0|   1900.0|
|        null|     null|
|       190.0|   2005.0|
|        94.0|   2009.0|
|        null|     null|
|        null|     null|
|        97.0|   1890.0|
|       110.0|   1880.0|
|       105.0|   1890.0|
|        null|     null|
|        73.0|   1985.0|
+------------+---------+
only showing top 20 rows



In [85]:
dataset2.describe().show()

+-------+----------+-------------+------------------+-----+-----------------+------+-------+---------+------------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------+-------------------+-------------------+--------------------+-----------------+
|summary|    Suburb|      Address|             Rooms| Type|            Price|Method|SellerG|     Date|          Distance|         Postcode|          Bedroom2|          Bathroom|               Car|         Landsize|      BuildingArea|         YearBuilt| CouncilArea|          Lattitude|         Longtitude|          Regionname|    Propertycount|
+-------+----------+-------------+------------------+-----+-----------------+------+-------+---------+------------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------+-------------------+-------------------+-----

In [44]:
dataset2.na.drop().show() #remove the rows having null values

+----------+-------------------+-----+----+---------+------+--------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|    Suburb|            Address|Rooms|Type|    Price|Method| SellerG|      Date|Distance|Postcode|Bedroom2|Bathroom|Car|Landsize|BuildingArea|YearBuilt|CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|
+----------+-------------------+-----+----+---------+------+--------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|Abbotsford|    25 Bloomburg St|    2|   h|1035000.0|     S|  Biggin| 4/02/2016|     2.5|  3067.0|     2.0|     1.0|0.0|   156.0|        79.0|   1900.0|      Yarra| -37.8079|  144.9934|Northern Metropol...|       4019.0|
|Abbotsford|       5 Charles St|    3|   h|1465000.0|    SP|  Biggin| 4/03/2017|     2.5|  3067.0|     3.0|     2.0|

In [46]:
dataset2.na.drop(how="any").show() #will drop rows with atleast one null

+----------+-------------------+-----+----+---------+------+--------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|    Suburb|            Address|Rooms|Type|    Price|Method| SellerG|      Date|Distance|Postcode|Bedroom2|Bathroom|Car|Landsize|BuildingArea|YearBuilt|CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|
+----------+-------------------+-----+----+---------+------+--------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|Abbotsford|    25 Bloomburg St|    2|   h|1035000.0|     S|  Biggin| 4/02/2016|     2.5|  3067.0|     2.0|     1.0|0.0|   156.0|        79.0|   1900.0|      Yarra| -37.8079|  144.9934|Northern Metropol...|       4019.0|
|Abbotsford|       5 Charles St|    3|   h|1465000.0|    SP|  Biggin| 4/03/2017|     2.5|  3067.0|     3.0|     2.0|

In [47]:
dataset2.na.drop(how="all").show() #will drop rows iff all null

+----------+-------------------+-----+----+---------+------+-------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|    Suburb|            Address|Rooms|Type|    Price|Method|SellerG|      Date|Distance|Postcode|Bedroom2|Bathroom|Car|Landsize|BuildingArea|YearBuilt|CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|
+----------+-------------------+-----+----+---------+------+-------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|Abbotsford|       85 Turner St|    2|   h|1480000.0|     S| Biggin| 3/12/2016|     2.5|  3067.0|     2.0|     1.0|1.0|   202.0|        null|     null|      Yarra| -37.7996|  144.9984|Northern Metropol...|       4019.0|
|Abbotsford|    25 Bloomburg St|    2|   h|1035000.0|     S| Biggin| 4/02/2016|     2.5|  3067.0|     2.0|     1.0|0.0| 

In [54]:
dataset2.na.drop(how="any",thresh=2).show() #atleast 2 non null values should be present to drop rows otherwise keep

+----------+-------------------+-----+----+---------+------+-------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|    Suburb|            Address|Rooms|Type|    Price|Method|SellerG|      Date|Distance|Postcode|Bedroom2|Bathroom|Car|Landsize|BuildingArea|YearBuilt|CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|
+----------+-------------------+-----+----+---------+------+-------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|Abbotsford|       85 Turner St|    2|   h|1480000.0|     S| Biggin| 3/12/2016|     2.5|  3067.0|     2.0|     1.0|1.0|   202.0|        null|     null|      Yarra| -37.7996|  144.9984|Northern Metropol...|       4019.0|
|Abbotsford|    25 Bloomburg St|    2|   h|1035000.0|     S| Biggin| 4/02/2016|     2.5|  3067.0|     2.0|     1.0|0.0| 

In [56]:
dataset2.na.drop(how="any",subset=['YearBuilt']).show() #drop null values from a specific column only

+----------+-------------------+-----+----+---------+------+--------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|    Suburb|            Address|Rooms|Type|    Price|Method| SellerG|      Date|Distance|Postcode|Bedroom2|Bathroom|Car|Landsize|BuildingArea|YearBuilt|CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|
+----------+-------------------+-----+----+---------+------+--------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|Abbotsford|    25 Bloomburg St|    2|   h|1035000.0|     S|  Biggin| 4/02/2016|     2.5|  3067.0|     2.0|     1.0|0.0|   156.0|        79.0|   1900.0|      Yarra| -37.8079|  144.9934|Northern Metropol...|       4019.0|
|Abbotsford|       5 Charles St|    3|   h|1465000.0|    SP|  Biggin| 4/03/2017|     2.5|  3067.0|     3.0|     2.0|

# Filling the missing values

In [86]:
dataset2.na.fill('Missing Values','YearBuilt').show()

+----------+-------------------+-----+----+---------+------+-------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|    Suburb|            Address|Rooms|Type|    Price|Method|SellerG|      Date|Distance|Postcode|Bedroom2|Bathroom|Car|Landsize|BuildingArea|YearBuilt|CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|
+----------+-------------------+-----+----+---------+------+-------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+
|Abbotsford|       85 Turner St|    2|   h|1480000.0|     S| Biggin| 3/12/2016|     2.5|  3067.0|     2.0|     1.0|1.0|   202.0|        null|     null|      Yarra| -37.7996|  144.9984|Northern Metropol...|       4019.0|
|Abbotsford|    25 Bloomburg St|    2|   h|1035000.0|     S| Biggin| 4/02/2016|     2.5|  3067.0|     2.0|     1.0|0.0| 

# Filling Missing values with Mean Imputation

In [90]:
from pyspark.ml.feature import Imputer
imputer = Imputer(inputCols=['BuildingArea','YearBuilt'],outputCols=["{}_imputed".format(c) for c in ['BuildingArea','YearBuilt']]).setStrategy("mean")

In [91]:
imputer.fit(dataset2).transform(dataset2).show()

+----------+-------------------+-----+----+---------+------+-------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+--------------------+------------------+
|    Suburb|            Address|Rooms|Type|    Price|Method|SellerG|      Date|Distance|Postcode|Bedroom2|Bathroom|Car|Landsize|BuildingArea|YearBuilt|CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|BuildingArea_imputed| YearBuilt_imputed|
+----------+-------------------+-----+----+---------+------+-------+----------+--------+--------+--------+--------+---+--------+------------+---------+-----------+---------+----------+--------------------+-------------+--------------------+------------------+
|Abbotsford|       85 Turner St|    2|   h|1480000.0|     S| Biggin| 3/12/2016|     2.5|  3067.0|     2.0|     1.0|1.0|   202.0|        null|     null|      Yarra| -37.7996|  144.9984|Northern Metropol...|       4019.0| 